In [8]:
import xml.etree.ElementTree as ET
import re
def parser(xmlInput):   # extracts pageid, title and text
    myroot = ET.fromstring(xmlInput)
    pageid, title, text = '', '', ''
    for x in myroot:
        if x.tag == 'revision':
            for y in x:
                if y.tag == 'text':
                    text = y.text
                    break;
        elif x.tag == 'id':
            pageid = x.text
        elif x.tag == 'title':
            title = x.text
    return pageid, title.strip().lower(), text

def parserRedirect(xmlInput):   # extracts redirect title if it exists
    myroot = ET.fromstring(xmlInput)
    title, redtitle = '', '#####'
    for x in myroot:    # traversing xml tree
        if x.tag == 'redirect':
            for y, z in x.attrib.items():
                if y == 'title':
                    redtitle = z
                    break
        elif x.tag == 'title':
            title = x.text
    return title.strip().lower(), redtitle.strip().lower()

def getLinksList(text):     # extracts wikilink references from text
    if text is None:
        return []
    ret = re.findall("\[\[[^\[\]\{\}:\\\/]+]]", text)    # matches [[pagename]], pagename does not contain ':', '\', '/'
    actret = []     # list to return
    for i in range(len(ret)):
        temp = ''
        for ch in ret[i]:
            if ch in ['|', ',', '#']:   # ignoring text after these because page title is complete
                break
            if ch != '[' and ch != ']':     # ignoring [[]]
                temp += ch
        temp = temp.lower().replace('\n', ' ')
        temp = ' '.join(temp.split())
        if temp != '':
            actret.append(temp)
    return actret

In [9]:
# Combines page titles to the main page that they redirect to
import bz2
alias = dict()
aliasfile = open('alias.txt', 'w')      # wrties output here
fd = bz2.open('./data/enwiki-latest-pages-articles.xml.bz2', 'r')
for i in range(1, 41):
    fd.readline()
cnt = 0
aliascount = 0
while True:
    xml = ''
    line = fd.readline().decode()
    cnt += 1
    if '<page>' not in line:  # end of file reached
        break
    while '</page>' not in line:
        xml += line.strip() + '\n'
        line = fd.readline().decode()
    xml += line.strip() + '\n'
    title, rdtitle = parserRedirect(xml)  # processing xml of the current page
    if title != '' and rdtitle != '' and rdtitle != '#####':  # page redirects to another page
        aliascount += 1
        alias[title] = rdtitle
        aliasfile.write(title + '\n' + rdtitle + '\n')      # write to alias.txt
    if cnt % 10000 == 0:    # progress update
        print(cnt, aliascount)
aliasfile.close()
fd.close()


10000 2430
20000 4747
30000 6721
40000 9766
50000 13327
60000 17278
70000 19670
80000 20125
90000 20564
100000 20764
110000 22740
120000 25802
130000 28978
140000 32919
150000 36908
160000 40820
170000 44696
180000 48943
190000 53448
200000 57691
210000 61678
220000 65853
230000 70462
240000 75309
250000 79609
260000 83941
270000 88291
280000 92485
290000 96428
300000 100643
310000 105041
320000 109895
330000 114924
340000 120000
350000 124585
360000 129611
370000 133894
380000 138172
390000 142634
400000 147182
410000 152779
420000 156447
430000 160493
440000 164481
450000 169552
460000 175076
470000 182282
480000 189183
490000 197414
500000 205683
510000 211915
520000 216005
530000 219953
540000 223870
550000 228239
560000 232348
570000 235781
580000 240077
590000 244498
600000 248708
610000 252755
620000 256873
630000 261240
640000 265137
650000 269453
660000 273010
670000 276945
680000 280552
690000 284800
700000 288966
710000 292701
720000 297003
730000 301321
740000 306491
750000

In [10]:
import bz2
fd = bz2.open('./data/enwiki-latest-pages-articles.xml.bz2', 'r')
for i in range(1, 41):
    fd.readline()
edgefile = open('edges.txt', 'w')
cnt = 0
lines = 0
pagenumber = dict()
while True:     # adds all edges to edges.txt
    xml = ''
    line = fd.readline().decode()
    if '<page>' not in line:    # end of file reached
        break
    while '</page>' not in line :
        xml += line.strip()+'\n'
        line = fd.readline().decode()
    xml += line.strip()+'\n'
    pageid, title, text = parser(xml)
    if re.fullmatch("[^\[\]\{\}:\\\/]+", title) is None:     # check if current page is valid
        continue
    links = getLinksList(text)  # get all links of the page
    edgefile.write(title+'\n')  # write the current page once
    lines += 1
    for link in links:  # write all children
        edgefile.write(link+'\n')
        lines += 1
    edgefile.write('###\n')     # current node done
    lines += 1
    if cnt % 10000 == 0:    # progress update
        print(cnt, lines)
    cnt += 1
print(lines)
fd.close()
edgefile.close()

0 3
10000 1839598
20000 3618839
30000 5168848
40000 6372408
50000 7503601
60000 8417358
70000 9277710
80000 9909568
90000 10616116
100000 11213769
110000 12078257
120000 13436534
130000 14297295
140000 15093329
150000 15853287
160000 16618937
170000 17267243
180000 17992092
190000 18622260
200000 19289299
210000 19974372
220000 20656917
230000 21217650
240000 21824279
250000 22426677
260000 23053216
270000 23644403
280000 24236864
290000 24766202
300000 25308758
310000 25760465
320000 26215569
330000 26650016
340000 27139803
350000 27591635
360000 28110699
370000 28611870
380000 29102103
390000 29583352
400000 29997731
410000 30497912
420000 30943282
430000 31332148
440000 31733821
450000 31927530
460000 32170469
470000 32321128
480000 32473241
490000 32807877
500000 33246765
510000 33698811
520000 34153701
530000 34586311
540000 35003867
550000 35413041
560000 35839209
570000 36228490
580000 36645554
590000 37034195
600000 37430206
610000 37811540
620000 38338944
630000 38774611
64000

In [1]:
adjlist = dict()
alias = dict()
nodeid = dict()
nodename = dict()
allnodes = dict()
visits = dict()
aliasfile = open('alias.txt', 'r')
edgefile = open('edges.txt', 'r')

def init():     # loads the graph into memory
    print('loading graph into memory')
    aliascount = 0
    while True:     # links pages that redirect to another page
        title = aliasfile.readline().strip()
        if title == '':
            break
        rdtitle = aliasfile.readline().strip()
        alias[title] = rdtitle
        aliascount += 1
        if aliascount % 10000 == 0:
            print('alias count: ' + str(aliascount))
    print('loading edges...')
    cnt = 0
    last = 0
    while True:     # loads edges into memory
        u = edgefile.readline().strip()
        if u == '':
            break
        if alias.get(u) is not None:    # if page redirects to another page
            u = alias[u]
        if nodeid.get(u) is None:   # assigning unique id to page for performance
            nodeid[u] = last
            nodename[last] = u
            last += 1
        cnt += 1
        u = nodeid[u]
        if adjlist.get(u) is None:
            adjlist[u] = []
        v = edgefile.readline().strip()
        cnt += 1
        while '###' not in v:   # no more edges for current node
            if cnt % 100000 == 0:   # progress update
                print('Edges added: ' + str(cnt))
            if v == '':     # ignoring blank lines
                cnt += 1
                v = edgefile.readline().strip()
                continue
            if alias.get(v) is not None:    # if page redirects to another page
                v = alias[v]
            if nodeid.get(v) is None:    # assigning unique id to page for performance
                nodeid[v] = last
                nodename[last] = v
                last += 1
            v = nodeid[v]
            adjlist[u].append(v)    # adding edge to adjacency list
            v = edgefile.readline().strip()
            cnt += 1
    aliasfile.close()
    edgefile.close()
    global allnodes
    allnodes = [c for c, c1 in adjlist.items()]     # list of all nodes

In [2]:
import random
def randomwalk(maxhops):    # runs a random walk of 'maxhops' steps
    hops = 0
    cur = random.choice(allnodes)   # starting node
    if visits.get(cur) is None:     # visit counter for page rank
        visits[cur] = 1
    else:
        visits[cur] += 1
    while hops < maxhops:
        if random.random() < 0.9:   # goes to a neighbour
            if adjlist.get(cur) is None or len(adjlist[cur]) == 0:
                cur = random.choice(allnodes)
            else:   # if cur has no children
                cur = random.choice(adjlist[cur])
        else:   # goes to a random node
            cur = random.choice(allnodes)
        if visits.get(cur) is None:
            visits[cur] = 1
        else:
            visits[cur] += 1
        hops += 1
        if hops % 1000000 == 0:     # progress tracker
            print(str(round((hops / maxhops * 100), 2)) + ' %')


def printlinks(ab):
    ab = ab.lower().strip()
    if alias.get(ab) is not None:
        ab = alias[ab]
    if adjlist.get(ab) is None:
        print('page not found')
        return
    for c in adjlist[nodeid[ab]]:
        print(nodename[c])


In [3]:
def main():
    init()
    while True:
        print('''Menu:
        1 : Run a random walk of X hops
        2 : Print the top K pages
        3 : See all links of a particular page
        Q : Quit'''
        )
        ip = input()
        if ip == '1':
            print('Enter number of hops: ')
            maxhops = int(input())
            randomwalk(maxhops)
        elif ip == '2':
            print('Enter number of top pages to be displayed')
            k = int(input())
            for a, b in sorted(visits.items(), key=lambda vk: (vk[1], vk[0]), reverse=True)[0:k]:
                print(nodename[a])
        elif ip == '3':
            print('Enter page name to look for: ')
            printlinks(input())
        elif ip == 'Q':
            print('Exiting')
            break
        else:
            print('Invalid input')


main()

loading graph into memory
alias count: 10000
alias count: 20000
alias count: 30000
alias count: 40000
alias count: 50000
alias count: 60000
alias count: 70000
alias count: 80000
alias count: 90000
alias count: 100000
alias count: 110000
alias count: 120000
alias count: 130000
alias count: 140000
alias count: 150000
alias count: 160000
alias count: 170000
alias count: 180000
alias count: 190000
alias count: 200000
alias count: 210000
alias count: 220000
alias count: 230000
alias count: 240000
alias count: 250000
alias count: 260000
alias count: 270000
alias count: 280000
alias count: 290000
alias count: 300000
alias count: 310000
alias count: 320000
alias count: 330000
alias count: 340000
alias count: 350000
alias count: 360000
alias count: 370000
alias count: 380000
alias count: 390000
alias count: 400000
alias count: 410000
alias count: 420000
alias count: 430000
alias count: 440000
alias count: 450000
alias count: 460000
alias count: 470000
alias count: 480000
alias count: 490000
ali

In [14]:
# Displays all countries according to their page rank on Wikipedia
pageranklist = sorted(visits.items(), key = lambda vk : (vk[1], vk[0]), reverse= True)
country = '''Afghanistan
Albania
Algeria
Andorra
Angola
Antigua and Barbuda
Argentina
Armenia
Australia
Austria
Azerbaijan
The Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia
Bosnia and Herzegovina
Botswana
Brazil
Brunei
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Cape Verde
Central African Republic
Chad
Chile
China
Colombia
Comoros
Republic of the Congo
Democratic Republic of the Congo
Costa Rica
Ivory Coast
Croatia
Cuba
Cyprus
Czech Republic
Denmark
Djibouti
Dominica
Dominican Republic
East Timor
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Fiji
Finland
France
Gabon
The Gambia
Georgia
Germany
Ghana
Greece
Grenada
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hungary
Iceland
India
Indonesia
Iran
Iraq
Ireland
Israel
Italy
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Kiribati
North Korea
South Korea
Kosovo
Kuwait
Kyrgyzstan
Laos
Latvia
Lebanon
Lesotho
Liberia
Libya
Liechtenstein
Lithuania
Luxembourg
Macedonia
Madagascar
Malawi
Malaysia
Maldives
Mali
Malta
Marshall Islands
Mauritania
Mauritius
Mexico
Federated States of Micronesia
Moldova
Monaco
Mongolia
Montenegro
Morocco
Mozambique
Myanmar
Namibia
Nauru
Nepal
Netherlands
New Zealand
Nicaragua
Niger
Nigeria
Norway
Oman
Pakistan
Palau
Panama
Papua New Guinea
Paraguay
Peru
Philippines
Poland
Portugal
Qatar
Romania
Russia
Rwanda
Saint Kitts and Nevis
Saint Lucia
Saint Vincent and the Grenadines
Samoa
San Marino
São Tomé and Príncipe
Saudi Arabia
Senegal
Serbia
Seychelles
Sierra Leone
Singapore
Slovakia
Slovenia
Solomon Islands
Somalia
South Africa
South Sudan
Spain
Sri Lanka
Sudan
Suriname
Swaziland
Sweden
Switzerland
Syria
Taiwan
Tajikistan
Tanzania
Thailand
Togo
Tonga
Trinidad and Tobago
Tunisia
Turkey
Turkmenistan
Tuvalu
Uganda
Ukraine
United Arab Emirates
United Kingdom
United States
Uruguay
Uzbekistan
Vanuatu
Vatican City
Venezuela
Vietnam
Yemen
Zambia
Zimbabwe'''
countrylist = []
for line in country.split('\n'):
    countrylist.append(line.strip().lower())
for a, b in pageranklist:
    if nodename[a] in countrylist:
        print(nodename[a], b)

In [17]:
# average and minimum hops required to go from page1 to page2 with a random walk
def randomwalkhops(cur):
    hops = 0
    while cur!= nodeid['tom cruise'.lower()]:
        if random.random() < 0.9:
            if adjlist.get(cur) is None or len(adjlist[cur]) == 0:
                cur = random.choice(allnodes)
            else:
                cur = random.choice(adjlist[cur])
        else:
            cur = random.choice(allnodes)
        hops += 1
    return hops
avghops = 0
minhops = 2e9
runs = 100 # number of random walks to conduct
startnode = 'tom hanks'     #set starting node
endnode = 'jerry seinfeld'      #set ending node
randomwalk(10000000)
for i in range(runs):
    startnode = startnode.strip().lower()
    endnode = endnode.strip().lower()
    if nodeid.get(startnode) is None:
        print(startnode + ' does not exist')
    elif nodeid.get(endnode) is None:
        print(endnode + ' does not exist')
    else:
        hops = randomwalkhops(startnode)
        avghops += hops/runs
        minhops = min(minhops, hops)
print(avghops, minhops)

10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
100.0 %
226643.62 641
